In [ ]:
# Install additional required libraries
!pip install tensorFlow
!pip install imbalanced-learn

In [ ]:
# Import required libraries

import pandas as pd
import numpy
import sys
sys.maxsize
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from imblearn.over_sampling import SMOTE

In [ ]:
# Import ANFIS implementation to be used
import myanfis

In [ ]:
# Import data
df = pd.read_csv('data.csv', header=[0])
df.head()

,VarName1,ORIGINE_1,ORIGINE_2,ORIGINE_3,SEXE_1,SEXE_2,IGSII_ADM_TYP_00,IGSII_ADM_TYP_60,IGSII_ADM_TYP_80,PATWGHT,...,PREBROINF0,PREBROINF1,PREBROINF2,PREBROINF3,PREBROINF4,PREBROINF5,PREBROINF6,PREBROINF7,ANTIBIOTIC,label
0,0,0,1,0,1,0,0,1,0,80,...,0,0,0,0,0,0,0,0,1,0
1,2,1,0,0,1,0,0,1,0,73,...,1,0,0,0,0,0,0,0,1,0
2,3,0,0,1,1,0,0,1,0,80,...,1,0,0,1,0,0,0,0,1,0
3,6,0,1,0,1,0,0,1,0,50,...,0,0,0,0,1,0,0,0,0,0
4,9,1,0,0,0,1,0,1,0,79,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
minmaxScaler = MinMaxScaler()

df2 = df.copy()

# Reduce data to best 13 feautures needed

chosen = ['AGE', 'SEXE_1', 'SEXE_2', 'LACTATES_J0', 'IGS3_ADM', 'My_AR_INF_Type', 'SOFA_INC2', 'Cort_DiffMax', 'GLYCEMIE', 'SOFA_ADM', 'ORIGINE_1', 'ORIGINE_2', 'ORIGINE_3', 'label']
for col in df2.columns:
    if col not in chosen:
        df2.drop(col, inplace=True, axis=1)

# Scaling features into 0-1

df2[df2.columns] = minmaxScaler.fit_transform(df2[df2.columns])
df2.head()

,ORIGINE_1,ORIGINE_2,ORIGINE_3,SEXE_1,SEXE_2,AGE,SOFA_INC2,SOFA_ADM,IGS3_ADM,GLYCEMIE,LACTATES_J0,Cort_DiffMax,My_AR_INF_Type,label
0,0.0,1.0,0.0,1.0,0.0,0.833333,0.40,0.391304,0.594406,0.217460,0.106667,0.173194,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.846154,0.45,0.608696,0.741259,0.242857,0.049333,0.159649,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.243590,0.25,0.130435,0.482517,0.296825,0.047994,0.147308,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.500000,0.65,0.391304,0.664336,0.160317,0.081333,0.137122,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.179487,0.35,0.521739,0.489510,0.228571,0.044000,0.179042,0.0,0.0


In [ ]:
# Split data into training and validation sets
#training
X_train = df2.iloc[:570,:13]
Y_train = df2.iloc[:570,-1]

# validation
X_test = df2.iloc[570:,:13]
Y_test = df2.iloc[570:,-1]

# Reduce overfitting and fix unbalanced data problem

sm = SMOTE(random_state=42)

X_sm, Y_sm = sm.fit_resample(X_train, Y_train)

print(f'''Shape of X before SMOTE: {X_train.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
Y_sm.value_counts(normalize=True) * 100

X_test, Y_test = sm.fit_resample(X_test, Y_test)

X_train=X_sm
Y_train=Y_sm

X_test.head()

Shape of X before SMOTE: (570, 13)
Shape of X after SMOTE: (716, 13)

Balance of positive and negative classes (%):


,ORIGINE_1,ORIGINE_2,ORIGINE_3,SEXE_1,SEXE_2,AGE,SOFA_INC2,SOFA_ADM,IGS3_ADM,GLYCEMIE,LACTATES_J0,Cort_DiffMax,My_AR_INF_Type
0,1.0,0.0,0.0,1.0,0.0,0.820513,0.50,0.434783,0.0,0.104762,0.102667,0.142391,0.0
1,0.0,1.0,0.0,1.0,0.0,0.628205,0.80,0.434783,0.0,0.247619,0.068000,0.132029,1.0
2,1.0,0.0,0.0,1.0,0.0,0.858974,0.40,0.391304,0.0,0.215873,0.030667,0.132029,1.0
3,1.0,0.0,0.0,0.0,1.0,0.974359,0.35,0.565217,0.0,0.134921,0.085333,0.132029,0.0
4,0.0,0.0,1.0,1.0,0.0,0.487179,0.70,0.652174,0.0,0.120635,0.074667,0.132029,0.0


In [ ]:
# Setting ANFIS parameters

param = myanfis.fis_parameters(
        n_input = 13,                # no. of Regressors
        n_memb = 2,                 # no. of fuzzy memberships
        batch_size = 1,
        memb_func = 'sigmoid',      # 'gaussian' / 'gbellmf' / 'sigmoid'
        optimizer = 'adam',          # sgd / adam / ...
        loss = tf.keras.losses.MeanSquaredError(),               # tf.keras.losses.MeanAbsoluteError(), mse / mae / huber_loss / mean_absolute_percentage_error / ...
        n_epochs = 500
        )

In [ ]:
# Calling ANFIS with parameters
fis = myanfis.ANFIS(n_input = param.n_input,
              n_memb = param.n_memb,
              batch_size = param.batch_size,
              memb_func = param.memb_func,
              name = 'firstAnfis'
              )

# Compile model
fis.model.compile(optimizer=param.optimizer,
                loss=param.loss,
                metrics=['accuracy']  # ['mae', 'mse']
                )

# Fit model
history = fis.fit(X_train, Y_train,
            epochs=param.n_epochs,
            batch_size=param.batch_size,
            #validation_split=0.1,
            validation_data = (X_test, Y_test),
            )
# Append training results to history
histories = []
histories.append(history)

Epoch 1/500
716/716 [==============================] - 5s 5ms/step - loss: 0.2811 - accuracy: 0.5056 - val_loss: 0.2609 - val_accuracy: 0.5227
Epoch 2/500
716/716 [==============================] - 3s 4ms/step - loss: 0.2459 - accuracy: 0.5237 - val_loss: 0.2394 - val_accuracy: 0.5682
Epoch 3/500
716/716 [==============================] - 3s 4ms/step - loss: 0.2304 - accuracy: 0.6131 - val_loss: 0.2255 - val_accuracy: 0.6591
Epoch 4/500
716/716 [==============================] - 3s 4ms/step - loss: 0.2209 - accuracy: 0.6383 - val_loss: 0.2168 - val_accuracy: 0.5909
Epoch 5/500
716/716 [==============================] - 3s 4ms/step - loss: 0.2160 - accuracy: 0.6662 - val_loss: 0.2094 - val_accuracy: 0.6818
Epoch 6/500
716/716 [==============================] - 3s 4ms/step - loss: 0.2112 - accuracy: 0.6620 - val_loss: 0.2015 - val_accuracy: 0.7045
Epoch 7/500
716/716 [==============================] - 3s 5ms/step - loss: 0.2082 - accuracy: 0.6676 - val_loss: 0.2013 - val_accuracy: 0.7500

In [ ]:
# Plot model performance

loss_curves = pd.DataFrame(history.history)
loss_curves.plot(figsize=(8, 5))

In [ ]:
# ------------------------- END ----------------------------